# Recurrent Neural Network

## Part 1 - Data Preprocessing

### Importing the libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
df= pd.read_csv('/content/drive/My Drive/project/out_0.1.csv')
print(df.head)

<bound method NDFrame.head of                                       run    type  ... attrvalue value
0       General-1-20210525-19:06:56-21400  scalar  ...       NaN     4
1       General-1-20210525-19:06:56-21400  scalar  ...       NaN     0
2       General-1-20210525-19:06:56-21400  scalar  ...       NaN    10
3       General-1-20210525-19:06:56-21400  scalar  ...       NaN     0
4       General-1-20210525-19:06:56-21400  scalar  ...       NaN     5
...                                   ...     ...  ...       ...   ...
876187  General-1-20210525-19:06:56-21400  scalar  ...       NaN     0
876188  General-1-20210525-19:06:56-21400  scalar  ...       NaN    14
876189  General-1-20210525-19:06:56-21400  scalar  ...       NaN     0
876190  General-1-20210525-19:06:56-21400  scalar  ...       NaN    15
876191  General-1-20210525-19:06:56-21400  scalar  ...       NaN     0

[876192 rows x 7 columns]>


### Importing the training set

In [ ]:

dataset = df
data = dataset.iloc[:,3:]
headings = list(data['name'])[0:2]
datadict = {headings[0]:list(data['value'])[0::2],headings[1]:list(data['value'])[1::2]}
newdata = pd.DataFrame(datadict)
print(newdata)

        #ONU ID  #Req size
0             4          0
1            10          0
2             5          0
3             8          0
4            12          0
...         ...        ...
438091       11          0
438092       12          0
438093       13          0
438094       14          0
438095       15          0

[438096 rows x 2 columns]


### Feature Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(newdata)
print(training_set_scaled)

[[0.26666667 0.        ]
 [0.66666667 0.        ]
 [0.33333333 0.        ]
 ...
 [0.86666667 0.        ]
 [0.93333333 0.        ]
 [1.         0.        ]]


### Creating a data structure with 60 timesteps and 1 output

In [ ]:
X_train = []
y_train = []
for i in range(60, 438096):
    X_train.append(training_set_scaled[i-60:i, :])
    y_train.append(training_set_scaled[i, :])
X_train, y_train = np.array(X_train), np.array(y_train)
#print(X_train.shape)

### Reshaping

In [ ]:
X_train = np.reshape(X_train,(X_train.shape[0], X_train.shape[1], 2))
print(X_train.shape)

(438036, 60, 2)


## Part 2 - Building and Training the RNN

### Importing the Keras libraries and packages

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

### Initialising the RNN

In [ ]:
regressor = Sequential()

### Adding the first LSTM layer and some Dropout regularisation

In [ ]:
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 2)))
regressor.add(Dropout(0.2))

### Adding a second LSTM layer and some Dropout regularisation

In [ ]:
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

### Adding a third LSTM layer and some Dropout regularisation

In [ ]:
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

### Adding a fourth LSTM layer and some Dropout regularisation

In [ ]:
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

### Adding the output layer

In [ ]:
regressor.add(Dense(units = 2))

### Compiling the RNN

In [ ]:
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

### Fitting the RNN to the Training set

In [ ]:
regressor.fit(X_train, y_train, epochs = 20, batch_size = 32)

## Part 3 - Making the predictions and visualising the results

### Getting the real stock price of 2017

In [ ]:
dataset_test = pd.read_csv('Google_Stock_Price_Test.csv')
real_stock_price = dataset_test.iloc[:, 1:2].values

### Getting the predicted stock price of 2017

In [ ]:
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, 80):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

### Visualising the results

In [ ]:
plt.plot(real_stock_price, color = 'red', label = 'Real Google Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Google Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()